<a href="https://colab.research.google.com/github/blmanvitha/stock-market-prediction/blob/main/stock_market_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance
!pip install tensorflow

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import tensorflow as tf

In [34]:
data = yf.download("GOOGL" , start = "2018-01-01" , interval = '1d')

[*********************100%***********************]  1 of 1 completed


In [ ]:
data.shape

(1307, 6)

In [ ]:
data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-02,52.651001,53.799000,52.651001,53.660500,53.660500,31766000
2018-01-03,53.696499,54.805000,53.671501,54.576000,54.576000,31318000
2018-01-04,54.854500,55.203999,54.713001,54.787998,54.787998,26052000
2018-01-05,55.172501,55.679001,55.090000,55.514500,55.514500,30250000
2018-01-08,55.549999,55.958000,55.500000,55.710499,55.710499,24644000


In [ ]:
data.sort_index(inplace = True)

In [ ]:
data = data.loc[~data.index.duplicated(keep='first')]

In [ ]:
data.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-03-07,94.980003,95.669998,93.529999,93.860001,93.860001,27835500
2023-03-08,94.120003,95.959999,94.000000,94.250000,94.250000,34103300
2023-03-09,94.050003,95.529999,91.900002,92.320000,92.320000,28813500
2023-03-10,92.169998,92.790001,90.400002,90.629997,90.629997,35927000
2023-03-13,90.089996,91.971298,89.419998,91.540001,91.540001,8188025


In [ ]:
data.describe() 

,Open,High,Low,Close,Adj Close,Volume
count,1307.000000,1307.000000,1307.000000,1307.000000,1307.000000,1.307000e+03
mean,86.004223,86.975270,85.039922,86.026196,86.026196,3.543396e+07
std,30.260418,30.556922,29.906850,30.225497,30.225497,1.613462e+07
min,49.216000,50.605999,48.882999,49.233501,49.233501,8.188025e+06
25%,59.021250,59.445749,58.391500,58.951750,58.951750,2.520400e+07
50%,75.074997,76.002502,74.485001,75.432999,75.432999,3.155800e+07
75%,111.399502,113.150501,110.139751,111.622749,111.622749,4.031000e+07
max,151.250000,151.546494,148.899002,149.838501,149.838501,1.331780e+08


In [ ]:
data.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [ ]:
import plotly.graph_objects as go

# Check the trend in Closing Values 
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = data['Close'] , mode = 'lines'))
fig.update_layout(height = 500 , width = 900, 
                  xaxis_title='Date' , yaxis_title='Close')
fig.show()

In [ ]:
# Check the trend in Volume Traded
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = data['Volume'] , mode = 'lines'))
fig.update_layout(height = 500 , width = 900, 
                  xaxis_title='Date' , yaxis_title='Volume')
fig.show()
     

In [ ]:
from sklearn.preprocessing import MinMaxScaler 
import pickle 
from tqdm.notebook import tnrange

In [ ]:
# Filter only required data 
data = data[['Close' , 'Volume']] 

In [ ]:
data.head()

,Close,Volume
Date,,
2018-01-02,53.660500,31766000
2018-01-03,54.576000,31318000
2018-01-04,54.787998,26052000
2018-01-05,55.514500,30250000
2018-01-08,55.710499,24644000


In [ ]:
# Confirm the Testing Set length 
test_length = data[(data.index >= '2020-09-01')].shape[0]

In [ ]:
def CreateFeatures_and_Targets(data, feature_length):
    X = []
    Y = []

    for i in tnrange(len(data) - feature_length): 
        X.append(data.iloc[i : i + feature_length,:].values)
        Y.append(data["Close"].values[i+feature_length])

    X = np.array(X)
    Y = np.array(Y)

    return X , Y

In [ ]:
X , Y = CreateFeatures_and_Targets(data , 32)

  0%|          | 0/1275 [00:00<?, ?it/s]

In [ ]:
# Check the shapes
X.shape , Y.shape

((1275, 32, 2), (1275,))

In [ ]:
Xtrain , Xtest , Ytrain , Ytest = X[:-test_length] , X[-test_length:] , Y[:-test_length] , Y[-test_length:]

In [ ]:
# Check Training Dataset Shape 
Xtrain.shape , Ytrain.shape

((639, 32, 2), (639,))

In [ ]:
# Check Testing Dataset Shape
Xtest.shape , Ytest.shape

((636, 32, 2), (636,))

In [ ]:
# Create a Scaler to Scale Vectors with Multiple Dimensions 
class MultiDimensionScaler():
    def __init__(self):
        self.scalers = []

    def fit_transform(self , X):
        total_dims = X.shape[2]
        for i in range(total_dims):
            Scaler = MinMaxScaler()
            X[:, :, i] = Scaler.fit_transform(X[:, :, i])
            self.scalers.append(Scaler)
        return X

    def transform(self , X):
        for i in range(X.shape[2]):
            X[:, :, i] = self.scalers[i].transform(X[:,:,i])
        return X 

In [ ]:
Feature_Scaler = MultiDimensionScaler()
Xtrain = Feature_Scaler.fit_transform(Xtrain)
Xtest = Feature_Scaler.transform(Xtest)    

In [ ]:
Target_Scaler = MinMaxScaler()
Ytrain = Target_Scaler.fit_transform(Ytrain.reshape(-1,1))
Ytest = Target_Scaler.transform(Ytest.reshape(-1,1))

In [ ]:
def save_object(obj , name : str):
    pickle_out = open(f"{name}.pck","wb")
    pickle.dump(obj, pickle_out)
    pickle_out.close()

def load_object(name : str):
    pickle_in = open(f"{name}.pck","rb")
    data = pickle.load(pickle_in)
    return data

In [ ]:
# Save your objects for future purposes 
save_object(Feature_Scaler , "Feature_Scaler")
save_object(Target_Scaler , "Target_Scaler")

In [36]:
from tensorflow.keras.callbacks import ModelCheckpoint , ReduceLROnPlateau

save_best = ModelCheckpoint("best_weights.h5", monitor='val_loss', save_best_only=True, save_weights_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.25,patience=4, min_lr=0.00001,verbose = 1)
     

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Dropout , LSTM , Bidirectional

model = Sequential()

model.add(Bidirectional(LSTM(512 ,return_sequences=True , recurrent_dropout=0.1, input_shape=(32, 2))))
model.add(LSTM(256 ,recurrent_dropout=0.1))
model.add(Dropout(0.3))
model.add(Dense(64 , activation='elu'))
model.add(Dropout(0.3))
model.add(Dense(32 , activation='elu'))
model.add(Dense(1 , activation='linear'))

In [ ]:
#optimizer = tf.keras.optimizers.Adam(learning_rate=0.002)
optimizer = tf.keras.optimizers.SGD(learning_rate = 0.002)
model.compile(loss='mse', optimizer=optimizer)

In [31]:
history = model.fit(Xtrain, Ytrain,epochs=10,batch_size = 1,verbose=1,shuffle=False ,validation_data=(Xtest , Ytest),callbacks=[reduce_lr])

Epoch 1/10
639/639 [==============================] - 1128s 2s/step - loss: 0.0098 - val_loss: 1.0047 - lr: 0.0020
Epoch 2/10
639/639 [==============================] - 1050s 2s/step - loss: 0.0119 - val_loss: 0.7072 - lr: 0.0020
Epoch 3/10
639/639 [==============================] - 1044s 2s/step - loss: 0.0102 - val_loss: 0.5662 - lr: 0.0020
Epoch 4/10
639/639 [==============================] - 1015s 2s/step - loss: 0.0071 - val_loss: 0.5099 - lr: 0.0020
Epoch 5/10
639/639 [==============================] - 1007s 2s/step - loss: 0.0066 - val_loss: 0.4296 - lr: 0.0020
Epoch 6/10
639/639 [==============================] - 990s 2s/step - loss: 0.0060 - val_loss: 0.3640 - lr: 0.0020
Epoch 7/10
639/639 [==============================] - 989s 2s/step - loss: 0.0055 - val_loss: 0.2805 - lr: 0.0020
Epoch 8/10
639/639 [==============================] - 985s 2s/step - loss: 0.0053 - val_loss: 0.2632 - lr: 0.0020
Epoch 9/10
639/639 [==============================] - 1099s 2s/step - loss: 0.0052 

In [32]:
# Checking the model Structure 
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (1, 32, 1024)            2109440   
 l)                                                              
                                                                 
 lstm_1 (LSTM)               (1, 256)                  1311744   
                                                                 
 dropout (Dropout)           (1, 256)                  0         
                                                                 
 dense (Dense)               (1, 64)                   16448     
                                                                 
 dropout_1 (Dropout)         (1, 64)                   0         
                                                                 
 dense_1 (Dense)             (1, 32)                   2080      
                                                        

In [38]:
Predictions = model.predict(Xtest)

20/20 [==============================] - 9s 363ms/step


In [39]:
Predictions = Target_Scaler.inverse_transform(Predictions)
Actual = Target_Scaler.inverse_transform(Ytest)

In [40]:
Predictions = np.squeeze(Predictions , axis = 1)
Actual = np.squeeze(Actual , axis = 1)

In [41]:
# Creating Sample Test Dataframe
test_dataframe_dict = {'Actual' : list(Actual) , 'Predicted' : list(Predictions)}
test_df = pd.DataFrame.from_dict(test_dataframe_dict)

test_df.index = data.index[-test_length:]

In [42]:
test_df.head()

,Actual,Predicted
Date,,
2020-08-31,82.753998,79.451889
2020-09-01,85.869499,79.830521
2020-09-02,81.475502,80.168823
2020-09-03,79.060501,80.118896
2020-09-04,76.180000,79.922325


In [43]:
# Check the trend in Volume Traded
fig = go.Figure()

fig.add_trace(go.Scatter(x = test_df.index , y = Actual , mode = 'lines' , name='Actual'))
fig.add_trace(go.Scatter(x = test_df.index , y = Predictions , mode = 'lines' , name='Predicted'))
fig.show()

In [44]:
Total_features = np.concatenate((Xtrain , Xtest) , axis = 0)

In [45]:
Total_Targets = np.concatenate((Ytrain , Ytest) , axis = 0)

In [46]:
Predictions = model.predict(Total_features)

40/40 [==============================] - 17s 429ms/step


In [47]:
Predictions = Target_Scaler.inverse_transform(Predictions)
Actual = Target_Scaler.inverse_transform(Total_Targets)

In [48]:
Predictions = np.squeeze(Predictions , axis = 1)
Actual = np.squeeze(Actual , axis = 1)

In [49]:
# Check the trend in Volume Traded
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = Actual , mode = 'lines' , name='Actual'))
fig.add_trace(go.Scatter(x = data.index , y = Predictions , mode = 'lines' , name='Predicted'))
fig.show()